In [ ]:
### Set up directory
import sys
import os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Check if directory exists, if not create it
save_path = os.path.join(parent_dir, 'generated_series')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os
from data.serialize import serialize_arr, deserialize_str, SerializerSettings
import pickle
from scipy.integrate import odeint


def serialize_gaussian(prec, time_series, mean_series, sigma_series, rescale_factor = 0.7, up_shift = 0.15, span = None):
    """
    Serialize a time series with gaussian noise and continuous support.
    span: range of raw time series

    Parameters:
    prec (int): Precision of the serialization
    time_series (list): The time series data
    mean_series (list): The mean series data
    sigma_series (list): The sigma series data

    Returns:
    tuple: A tuple containing 
        serialized time series: str
        rescaled mean series: np array
        rescaled sigma series: np array
    """
    settings=SerializerSettings(base=10, prec=prec, signed=True, time_sep=',', bit_sep='', minus_sign='-', fixed_length=False, max_val = 10)
    time_series = np.array(time_series)
    ### Final range is from 0.15 to 0.85

    if span is None:
        span = time_series.max()-time_series.min()
        
    rescaled_array = (time_series-time_series.min())/span * rescale_factor + up_shift
    rescaled_true_mean_arr = (np.array(mean_series)-time_series.min())/span * rescale_factor + up_shift
    rescaled_true_sigma_arr = np.array(sigma_series)/span * rescale_factor 
    rescaled_true_mean_arr *= 10
    rescaled_true_sigma_arr *= 10
    rescaled_full_series = 10*rescaled_array
    # print(rescaled_array)
    assert all(rescaled_array >= 0) and all(rescaled_array <= 1), "Elements in rescaled_array should be between 0 and 1"
    # print(min(rescaled_array))
    full_series = serialize_arr(rescaled_array, settings)
    return (rescaled_full_series, full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr)

def generate_transition_matrix(N_state):
    """
    Generate a random transition matrix of shape (N_state, N_state).
    Each row sums to 1.
    """
    P = np.random.rand(N_state, N_state)
    P /= P.sum(axis=1)[:, np.newaxis]
    return P

def generate_gaussian_matrix(N_state, sigma = 0.5):
    """
    Generate a transition matrix of shape (N_state, N_state) for an uncorrelated Brownian motion.
    Each row will be the same, representing discretized bins of a Gaussian distribution.
    The sum of each row is normalized to 1.

    :param N_state: Number of states (and thus size of the transition matrix)
    :param sigma: Standard deviation of the Gaussian distribution (controls the width of the Gaussian)
    """
    # Define the bins for the Gaussian distribution
    bins = np.linspace(-3, 3, N_state)
    
    # Calculate the Gaussian distribution values for these bins
    gaussian_distribution = np.exp(-0.5 * (bins / sigma) ** 2)
    gaussian_distribution /= gaussian_distribution.sum()  # Normalize to sum to 1

    # Repeat this distribution for each row to create the transition matrix
    P = np.tile(gaussian_distribution, (N_state, 1))

    return P

### Generate Series with Continuous Support

In [ ]:
num_series = 2
llama_size = '70b'

mode = 'all'
prec = 2
refine_depth = 1

kernel = 'NA'


In [ ]:
num_series = 2
llama_size = 'KDE'
kernel = 'gaussian'

mode = 'all'
prec = 2
refine_depth = 1


In [ ]:
num_series = 2
llama_size = 'histogram'
kernel = 'NA'

mode = 'all'
prec = 2
refine_depth = 1

In [ ]:

### Uncorrelated Gaussian'
sigma = 0.5
# sigma = 0.8
# sigma = 0.1
# sigma = 0.3
Nt = 400
traj_name = f'uncorrelated_gaussian_centered_sigma_{sigma}_{llama_size}'

for traj_idx in range(num_series):
    random_seed = traj_idx+2
    np.random.seed(random_seed)
    time_series = []
    mean_series = []
    sigma_series = []
    for t in range(0, Nt):
        W =  np.random.normal() * sigma 
        
        time_series.append(W)
        mean_series.append(0)
        sigma_series.append(sigma)
    
    rescale_factor = 0.2
    span = 0.2*7
    up_shift = 0.5 + min(time_series)/span*rescale_factor
    
    rescaled_full_series, full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series, rescale_factor = rescale_factor, up_shift = up_shift, span = span)
    # print(max(time_series)-min(time_series))
    # print(rescaled_true_sigma_arr[1])    
    # print(rescaled_true_mean_arr[1])    
    plt.figure(figsize=(10, 5))
    plt.plot((np.array(time_series)-min(time_series))/span*rescale_factor+up_shift, lw = 0, marker = 'o', alpha = 1)
    plt.ylim(0,1)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series, # string format for LLM
            'rescaled_full_series': rescaled_full_series, # double format for baseline
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'kernel': kernel,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
### Uncorrelated uniform dist.
# for uniform distribution, sigma = half width
# sigma = 0.8
# sigma = 0.5
# sigma = 0.3
sigma = 0.1
Nt = 400
traj_name = f'uncorrelated_uniform_centered_sigma_{sigma}'

for traj_idx in range(num_series):
    random_seed = traj_idx+2
    np.random.seed(random_seed)
    time_series = []
    mean_series = []
    sigma_series = []
    for t in range(0, Nt):
        W =  np.random.uniform(low=-sigma, high=sigma)
        
        time_series.append(W)
        mean_series.append(0)
        sigma_series.append(sigma)
    
    rescale_factor = 0.5
    span = 0.2*7
    up_shift = 0.5 + min(time_series)/span*rescale_factor
    
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series, rescale_factor = rescale_factor, up_shift = up_shift, span = span)  
    plt.figure(figsize=(10, 5))
    plt.plot((np.array(time_series)-min(time_series))/span*rescale_factor+up_shift, lw = 0, marker = 'o', alpha = 1)
    plt.ylim(0,1)
    # Save the generated data to a dictionary
    data_dict = {
            'dist type': "uniform",
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
####################################
### Generate continuous time series
####################################

### Brownian Motion



traj_name = 'brownian_motion'
Nt = 1000 # number of steps
a = 0 # drift

# other hyper-parameters
dt =  0.2 # time step
tspan = np.linspace(0, Nt*dt, Nt)
sigma = 0.8

for traj_idx in range(num_series):
    random_seed = 7+traj_idx
    print("random_seed: ", random_seed)
    np.random.seed(random_seed)
    # Initialize the time series
    x = 0  # Starting point
    time_series = [x]
    mean_series = [x]
    sigma_series = [0]

    # Generate the drift-diffusion time series
    for t in range(1, Nt):
        x_mean = x + a*dt
        x_sigma = sigma * np.sqrt(dt)
        dW =  np.random.normal()  # Wiener process (Brownian motion)
        x = x_mean + x_sigma * dW
        
        time_series.append(x)
        mean_series.append(x_mean)
        sigma_series.append(x_sigma)
        
    # plt.figure(figsize=(10, 5))
    # plt.plot(time_series, alpha = 1)

    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
        'full_series': full_series,
        'rescaled_true_mean_arr': rescaled_true_mean_arr,
        'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
        'prec': prec,
        'llama_size': llama_size,
        'mode': mode,
        'refine_depth': refine_depth,
        'random_seed': random_seed,
        'time_series': np.array(time_series)
    }

    # Pickle and save the dictionary
    # Count existing files in the save_path and assign a new number
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)


In [ ]:


traj_name = 'correlated_gaussian'
Nt = 1000 # number of steps
a = 0 # drift
theta = 0.5 # restoring spring constant

# other hyper-parameters
dt =  0.2 # time step
tspan = np.linspace(0, Nt*dt, Nt)
sigma = 0.8

for traj_idx in range(num_series):
    random_seed = traj_idx
    print("random_seed: ", random_seed)
    np.random.seed(random_seed)
    # Initialize the time series
    x = 0  # Starting point
    time_series = [x]
    mean_series = [x]
    sigma_series = [0]

    # Generate the drift-diffusion time series
    for t in range(1, Nt):
        x_mean = x + a*dt - x * theta*dt
        x_sigma = sigma * np.sqrt(dt)
        dW =  np.random.normal()  # Wiener process (Brownian motion)
        x = x_mean + x_sigma * dW
        
        time_series.append(x)
        mean_series.append(x_mean)
        sigma_series.append(x_sigma)
        
    plt.figure(figsize=(10, 5))
    plt.plot(time_series, alpha = 1)

    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
        'full_series': full_series,
        'rescaled_true_mean_arr': rescaled_true_mean_arr,
        'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
        'prec': prec,
        'llama_size': llama_size,
        'mode': mode,
        'refine_depth': refine_depth,
        'random_seed': random_seed,
        'time_series': np.array(time_series)
    }

    # Pickle and save the dictionary
    # Count existing files in the save_path and assign a new number
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)


In [ ]:
### Gemetric Brownian Motion
traj_name = 'geometric_brownian_motion'
# hyper-parameter to load from json


Nt = 1000 # number of steps
sigma = 0.6
a = None

if a is None:
    a = sigma**2/2
    
dt =  0.2 # time step
tspan = np.linspace(1, Nt*dt, Nt)

for traj_idx in range(num_series):
    random_seed = traj_idx+2
    np.random.seed(random_seed)

    # Initialize the time series
    x = 10  # Starting point
    time_series = [x]
    mean_series = [x]
    sigma_series = [0]

    # Generate the drift-diffusion time series
    for t in range(1, Nt):
        x_mean = x + a*dt
        x_sigma = x * sigma * np.sqrt(dt)
        dW =  np.random.normal()  # Wiener process (Brownian motion)
        x = x_mean + x_sigma * dW
        
        time_series.append(x)
        mean_series.append(x_mean)
        sigma_series.append(x_sigma)
        
    # plt.figure(figsize=(10, 5))
    # plt.plot(time_series, alpha = 1)
    # plt.yscale('log')
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
        # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }

    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)



In [ ]:
### Noisy Logistic Map
traj_name = 'noisy_logistic_map'
# hyper-parameter to load from json


Nt = 1000
sigma = 0.0136
r = 3.9

### Logistic Map
def logistic_map(r, x):
    return r * x * (1 - x)

### Manual differentiation
def logistic_map_diff(r, x):
    return np.abs(r * (1-2*x))

counter = -1
traj_idx = 3
while counter < num_series-1:
    random_seed = traj_idx
    np.random.seed(random_seed)
    traj_idx += 1
    x = 0.5 
    x_diff = 0 # Initial uncertainty is zero
    time_series = []
    mean_series = []
    sigma_series = []
        
    for i in range(Nt):
        x_mean = logistic_map(r, x)
        x_sigma = x_diff * sigma
        x = logistic_map(r, x + sigma * np.random.normal())
        x_diff = logistic_map_diff(r, x)
        
        time_series.append(x)
        mean_series.append(x_mean)
        sigma_series.append(x_sigma)
    
    if np.any(np.abs(np.array(time_series))>1):
        # discard stray trajectories
        continue
    else:
        print(f"good seed: {traj_idx-1}")
        counter += 1
        
    plt.plot(time_series, alpha = 1, lw = 0, marker = 'o')
    plt.show()
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
### Logistic Map
traj_name = 'logistic_map'
# hyper-parameter to load from json

Nt = 1000
sigma = 0
r = 3.9

### Logistic Map
def logistic_map(r, x):
    return r * x * (1 - x)

### Manual differentiation
def logistic_map_diff(r, x):
    return np.abs(r * (1-2*x))

counter = -1
traj_idx = 0
while counter < num_series-1:
    random_seed = traj_idx
    np.random.seed(random_seed)
    traj_idx += 1
    x = np.random.uniform() * 0.7
    x_diff = 0 # Initial uncertainty is zero
    time_series = []
    mean_series = []
    sigma_series = []
        
    for i in range(Nt):
        x_mean = logistic_map(r, x)
        x_sigma = x_diff * sigma
        x = logistic_map(r, x + sigma * np.random.normal())
        x_diff = logistic_map_diff(r, x)
        
        time_series.append(x)
        mean_series.append(x_mean)
        sigma_series.append(x_sigma)
    
    if np.any(np.abs(np.array(time_series))>1):
        # discard stray trajectories
        continue
    else:
        print(f"good seed: {traj_idx-1}")
        counter += 1
        
    plt.plot(time_series, alpha = 1, lw = 0, marker = 'o')
    plt.show()
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
traj_name = 'lorenz_system'
# hyper-parameter to load from json

# Nt = 1000
# Nt = 300
Nt = 400
tspan = np.linspace(0, 100, Nt)
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

def lorenz(state, t):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

counter = -1
traj_idx = 0
for i in range(num_series):
    # Specific initial condition
    
    random_seed = i
    np.random.seed(random_seed)
    initial_state = [np.random.uniform()*0.3, 2.01, 23.02] 
    
    solution = odeint(lorenz, initial_state, tspan)
    X = solution[:, 0]
    time_series = X.tolist()
    mean_series = time_series
    sigma_series = [0] * Nt
    
        
    plt.plot(time_series, alpha = 1, lw = 1, marker = 'o')
    plt.show()
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
### Uncorrelated Gaussian
traj_name = 'uncorrelated_gaussian'


Nt = 1000
sigma = 1
for traj_idx in range(num_series):
    random_seed = traj_idx+2
    np.random.seed(random_seed)
    time_series = []
    mean_series = []
    sigma_series = []
    for t in range(0, Nt):
        W =  np.random.normal() * sigma 
        
        time_series.append(W)
        mean_series.append(0)
        sigma_series.append(sigma)
        
    # plt.figure(figsize=(10, 5))
    # plt.plot(time_series, lw = 0, marker = 'o', alpha = 1)
    
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

In [ ]:
### Uncorrelated Uniform
traj_name  = 'uncorrelated_uniform'
# hyper-parameter to load from json

Nt = 1000 

sigma = 1
for traj_idx in range(num_series):
    random_seed = traj_idx
    np.random.seed(random_seed)
    time_series = []
    mean_series = []
    sigma_series = []
    for t in range(0, Nt):
        W =  np.random.uniform() * sigma 
        
        time_series.append(W)
        # mean_series.append(0)
        # sigma_series.append(sigma)
        
    # plt.figure(figsize=(10, 5))
    # plt.plot(time_series, lw = 0, marker = 'o', alpha = 1)
    
    full_series, rescaled_true_mean_arr, rescaled_true_sigma_arr = serialize_gaussian(prec, time_series, mean_series, sigma_series)
    # Save the generated data to a dictionary
    data_dict = {
            'full_series': full_series,
            'rescaled_true_mean_arr': rescaled_true_mean_arr,
            'rescaled_true_sigma_arr': rescaled_true_sigma_arr,
            'prec': prec,
            'llama_size': llama_size,
            'mode': mode,
            'refine_depth': refine_depth,
            'random_seed': random_seed,
            'time_series': np.array(time_series)
        }
    
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)

### Generate Markov Chains

In [ ]:
time_sep = ""

In [ ]:
llama_size_list = ['13b','13b','13b','13b','13b','13b']
Nt_list = [1500, 1500, 1500, 1500, 1500, 1500, 1500]
N_state_list = [4,5,6,7,8,9,10]
random_seed_list = [1,1,1,1,1,1,1]

llama_size_list += ['70b','70b','70b']
Nt_list += [1500, 1500, 1500]
N_state_list += [8,9,10]
random_seed_list += [1,1,1]

In [ ]:
llama_size_list = ['13b','13b','13b','13b','13b','13b','13b','13b','13b']
Nt_list = [1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500]
N_state_list = [4] * 9
random_seed_list = [2,3,4,5,6,7,8,9,10]

In [ ]:
traj_name  = 'markov_chain'
for llama_size, Nt, N_state, random_seed in zip(llama_size_list, Nt_list, 
                                                N_state_list, random_seed_list):
    states = np.arange(N_state)
    chain = [0]
    np.random.seed(random_seed)
    P = generate_transition_matrix(N_state)
        
    for t in range(1, Nt):
        current_state = chain[-1]
        next_state = np.random.choice(states, p=P[current_state])
        chain.append(next_state)
    
    # Convert the chain list to a string and store it in full_series
    full_series = "".join(str(x) for x in chain)
    
    data_dict = {
            'full_series': full_series,
            'full_array': np.array(chain),
            'llama_size': llama_size,
            
            'random_seed': random_seed,
            'P': P
        }
    # Pickle and save the dictionary
    file_indices = [int(name[:-4].rsplit('_',1)[1]) for name in os.listdir(save_path) if os.path.isfile(os.path.join(save_path, name)) and name.startswith(traj_name)]
    file_indices += [-1]
    save_name = os.path.join(save_path, f'{traj_name}_{max(file_indices)+1}.pkl')
    with open(save_name, 'wb') as f:
        pickle.dump(data_dict, f)